In [6]:
from functools import wraps
import logging
logger = logging.getLogger()

class ControlledException(Exception):
    pass

RETRIES_LIMIT = 3

def with_retry(retries_limit: int = RETRIES_LIMIT, allowed_exceptions: Exception = None):
    allowed_exception = allowed_exceptions or (ControlledException,)

    def retry(operation):
        @wraps(operation)
        def wrapped(*args, **kargs):
            last_raised = None
            for _ in range(retries_limit):
                try:
                    return operation(*args, **kargs)
                except allowed_exceptions as e:
                    logger.info('retrying %s due to %s', operation, e)
                    print('error')
                    last_raised = e
            raise last_raised
        return wrapped

    return retry

@with_retry(retries_limit= 5, allowed_exceptions=ControlledException)
def printHello():
    raise ControlledException("error!!")

printHello()     


error
error
error
error
error


ControlledException: error!!

In [10]:
class WithRetry:
    def __init__(self, retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
        self.retries_limit = retries_limit
        self.allowed_exception = allowed_exceptions

    def __call__(self, operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised= None
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exception as e:
                    last_raised = e
                    print('error!')
            raise last_raised
        return wrapped

@WithRetry(retries_limit=10,allowed_exceptions=ControlledException)
def printError():
    raise ControlledException("error!!!")

printError()

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!


ControlledException: error!!!